### Data Dictionary

The following exercises all work with the movielens data, which can be loaded using the following code:

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

%matplotlib inline
sns.set_style('dark')
sns.set(font_scale=1.2)

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, mean_squared_error,r2_score
from sklearn.metrics import plot_confusion_matrix, plot_precision_recall_curve, plot_roc_curve, accuracy_score
from sklearn.metrics import auc, f1_score, precision_score, recall_score, roc_auc_score

import warnings
warnings.filterwarnings('ignore')

# from pycaret.classification import *
# from pycaret.clustering import *
# from pycaret.regression import *

pd.options.display.max_columns= None
#pd.options.display.max_rows = None

### Data Exploration

In [2]:
df = pd.read_csv("movielens.csv", encoding='ISO-8859-1')

In [3]:
df

,movieId,title,year,genres,userId,rating,timestamp
0,31,Dangerous Minds,1995.0,Drama,1,2.5,1260759144
1,1029,Dumbo,1941.0,Animation|Children|Drama|Musical,1,3.0,1260759179
2,1061,Sleepers,1996.0,Thriller,1,3.0,1260759182
3,1129,Escape from New York,1981.0,Action|Adventure|Sci-Fi|Thriller,1,2.0,1260759185
4,1172,Cinema Paradiso (Nuovo cinema Paradiso),1989.0,Drama,1,4.0,1260759205
...,...,...,...,...,...,...,...
99999,6268,Raising Victor Vargas,2002.0,Comedy|Drama|Romance,671,2.5,1065579370
100000,6269,Stevie,2002.0,Documentary,671,4.0,1065149201
100001,6365,"Matrix Reloaded, The",2003.0,Action|Adventure|Sci-Fi|Thriller|IMAX,671,4.0,1070940363
100002,6385,Whale Rider,2002.0,Drama,671,2.5,1070979663


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    100004 non-null  int64  
 1   title      99997 non-null   object 
 2   year       99997 non-null   float64
 3   genres     100004 non-null  object 
 4   userId     100004 non-null  int64  
 5   rating     100004 non-null  float64
 6   timestamp  100004 non-null  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 5.3+ MB


In [5]:
df.describe(include='all')

,movieId,title,year,genres,userId,rating,timestamp
count,100004.000000,99997,99997.000000,100004,100004.000000,100004.000000,1.000040e+05
unique,NaN,8831,NaN,901,NaN,NaN,NaN
top,NaN,Forrest Gump,NaN,Drama,NaN,NaN,NaN
freq,NaN,341,NaN,7757,NaN,NaN,NaN
mean,12548.664363,NaN,1991.827305,NaN,347.011310,3.543608,1.129639e+09
std,26369.198969,NaN,15.088703,NaN,195.163838,1.058064,1.916858e+08
min,1.000000,NaN,1902.000000,NaN,1.000000,0.500000,7.896520e+08
25%,1028.000000,NaN,1987.000000,NaN,182.000000,3.000000,9.658478e+08
50%,2406.500000,NaN,1995.000000,NaN,367.000000,4.000000,1.110422e+09
75%,5418.000000,NaN,2001.000000,NaN,520.000000,4.000000,1.296192e+09


In [6]:
df.shape

(100004, 7)

In [7]:
df.columns

Index(['movieId', 'title', 'year', 'genres', 'userId', 'rating', 'timestamp'], dtype='object')

In [8]:
#Compute the number of ratings for each movie and then plot it against the year the movie came out. Use the square root transformation on the counts.
#What year has the highest median number of ratings?

df.groupby(by=['rating','title']).count()

movieId  year  genres  userId  \
rating title                                                            
0.5    "Great Performances" Cats              1     1       1       1   
       'Neath the Arizona Skies               1     1       1       1   
       'Round Midnight                        1     1       1       1   
       'Til There Was You                     1     1       1       1   
       (500) Days of Summer                   1     1       1       1   
...                                         ...   ...     ...     ...   
5.0    Zootopia                               3     3       3       3   
       Zorba the Greek (Alexis Zorbas)        2     2       2       2   
       Zorro, the Gay Blade                   1     1       1       1   
       eXistenZ                               4     4       4       4   
       ¡Three Amigos!                         1     1       1       1   

                                        timestamp  
rating title                                       
0.5    "Great Performances" Cats                1  
       'Neath the Arizona Skies                 1  
       'Round Midnight                          1  
       'Til There Was You                       1  
       (500) Days of Summer                     1  
...                                           ...  
5.0    Zootopia                                 3  
       Zorba the Greek (Alexis Zorbas)          2  
       Zorro, the Gay Blade                     1  
       eXistenZ                                 4  
       ¡Three Amigos!                           1  

[28292 rows x 5 columns]

In [9]:
df['rating'].value_counts()

4.0    28750
3.0    20064
5.0    15095
3.5    10538
4.5     7723
2.0     7271
2.5     4449
1.0     3326
1.5     1687
0.5     1101
Name: rating, dtype: int64

In [10]:
highrating = df[df['rating'] == 4.0]
highrating

,movieId,title,year,genres,userId,rating,timestamp
4,1172,Cinema Paradiso (Nuovo cinema Paradiso),1989.0,Drama,1,4.0,1260759205
12,1953,"French Connection, The",1971.0,Action|Crime|Thriller,1,4.0,1260759191
13,2105,Tron,1982.0,Action|Adventure|Sci-Fi,1,4.0,1260759139
20,10,GoldenEye,1995.0,Action|Adventure|Thriller,2,4.0,835355493
23,47,Seven (a.k.a. Se7en),1995.0,Mystery|Thriller,2,4.0,835355552
...,...,...,...,...,...,...,...
99992,5816,Harry Potter and the Chamber of Secrets,2002.0,Adventure|Fantasy,671,4.0,1065111963
99995,5989,Catch Me If You Can,2002.0,Crime|Drama,671,4.0,1064890625
99997,5995,"Pianist, The",2002.0,Drama|War,671,4.0,1066793014
100000,6269,Stevie,2002.0,Documentary,671,4.0,1065149201


In [11]:
highrating['year'].value_counts()

1995.0    1922
1994.0    1656
1996.0    1481
1999.0    1419
1993.0    1273
          ... 
1923.0       2
1921.0       2
1918.0       1
1917.0       1
1902.0       1
Name: year, Length: 99, dtype: int64

In [12]:
#We see that, on average, movies that came out after 1993 get more ratings. We also see that with newer movies, starting in 1993, the number of ratings decreases with year: the more recent a movie is, the less time users have had to rate it.

#Among movies that came out in 1993 or later, select the top 25 movies with the highest average number of ratings per year (n/year), and caculate the average rating of each of them. To calculate number of ratings per year, use 2018 as the end year.

In [13]:
df['year'].value_counts()

1995.0    6635
1994.0    5861
1996.0    5109
1999.0    5034
1993.0    4214
          ... 
1918.0       2
1917.0       2
1915.0       2
1919.0       1
1916.0       1
Name: year, Length: 103, dtype: int64

In [14]:
movies1993later = df[df['year'] > 1992]
movies1993later

,movieId,title,year,genres,userId,rating,timestamp
0,31,Dangerous Minds,1995.0,Drama,1,2.5,1260759144
2,1061,Sleepers,1996.0,Thriller,1,3.0,1260759182
11,1405,Beavis and Butt-Head Do America,1996.0,Adventure|Animation|Comedy|Crime,1,1.0,1260759203
16,2294,Antz,1998.0,Adventure|Animation|Children|Comedy|Fantasy,1,2.0,1260759108
20,10,GoldenEye,1995.0,Action|Adventure|Thriller,2,4.0,835355493
...,...,...,...,...,...,...,...
99999,6268,Raising Victor Vargas,2002.0,Comedy|Drama|Romance,671,2.5,1065579370
100000,6269,Stevie,2002.0,Documentary,671,4.0,1065149201
100001,6365,"Matrix Reloaded, The",2003.0,Action|Adventure|Sci-Fi|Thriller|IMAX,671,4.0,1070940363
100002,6385,Whale Rider,2002.0,Drama,671,2.5,1070979663


In [15]:
movies1993later.groupby(by=['year','title']).mean()

movieId      userId    rating  \
year   title                                                          
1993.0 ...And God Spoke                6600.0  394.000000  1.000000   
       Addams Family Values             410.0  328.863014  3.150685   
       Adventures of Huck Finn, The    5076.0  255.000000  3.500000   
       Age of Innocence, The            412.0  365.120000  3.540000   
       Alive                           3250.0  299.083333  2.750000   
...                                       ...         ...       ...   
2016.0 The Shallows                  158528.0   15.000000  3.500000   
       Warcraft                      122890.0  121.666667  4.000000   
       X-Men: Apocalypse             122924.0  119.333333  3.000000   
       Zoolander 2                   148888.0   98.000000  3.000000   
       Zootopia                      152081.0  284.444444  4.000000   

                                        timestamp  
year   title                                       
1993.0 ...And God Spoke              1.297549e+09  
       Addams Family Values          9.641278e+08  
       Adventures of Huck Finn, The  1.236981e+09  
       Age of Innocence, The         9.521940e+08  
       Alive                         9.860896e+08  
...                                           ...  
2016.0 The Shallows                  1.467259e+09  
       Warcraft                      1.466318e+09  
       X-Men: Apocalypse             1.469769e+09  
       Zoolander 2                   1.459405e+09  
       Zootopia                      1.468976e+09  

[5664 rows x 4 columns]

In [19]:
#What is the average rating for the movie The Shawshank Redemption?
movies1993later[movies1993later['movieId'] == 318].describe()

,movieId,year,userId,rating,timestamp
count,311.0,311.0,311.000000,311.000000,3.110000e+02
mean,318.0,1994.0,340.948553,4.487138,1.126809e+09
std,0.0,0.0,193.456115,0.702412,2.142776e+08
min,318.0,1994.0,3.000000,1.000000,8.282131e+08
25%,318.0,1994.0,163.500000,4.000000,9.446399e+08
50%,318.0,1994.0,350.000000,5.000000,1.117842e+09
75%,318.0,1994.0,510.500000,5.000000,1.309934e+09
max,318.0,1994.0,671.000000,5.000000,1.473804e+09


In [20]:
#What is the average number of ratings per year for the movie Forrest Gump?
movies1993later[movies1993later['movieId'] == 356].describe()

,movieId,year,userId,rating,timestamp
count,341.0,341.0,341.000000,341.000000,3.410000e+02
mean,356.0,1994.0,334.595308,4.054252,1.105382e+09
std,0.0,0.0,194.589070,0.871521,2.191051e+08
min,356.0,1994.0,2.000000,1.000000,8.327734e+08
25%,356.0,1994.0,161.000000,3.500000,8.980960e+08
50%,356.0,1994.0,333.000000,4.000000,1.097770e+09
75%,356.0,1994.0,508.000000,5.000000,1.299447e+09
max,356.0,1994.0,671.000000,5.000000,1.476478e+09


In [21]:
#Compute the average rating for each week and plot this average against date.

In [22]:
#The movielens data also has a genres column. This column includes every genre that applies to the movie. Some movies fall under several genres. Define a category as whatever combination appears in this column. Keep only categories with more than 1,000 ratings. Then compute the average and standard error for each category. Plot these as error bar plots.

In [26]:
genres = df.groupby(by='genres', as_index=False).sum()
genres

,genres,movieId,year,userId,rating,timestamp
0,(no genres listed),2385483,32116.0,7391,68.0,26088165855
1,Action,2070878,285261.0,52021,412.0,153870218573
2,Action|Adventure,2889493,1073155.0,182730,2046.0,620972100175
3,Action|Adventure|Animation,2521780,46133.0,4427,91.5,32973143465
4,Action|Adventure|Animation|Children,1440408,30139.0,5350,53.5,21361144336
...,...,...,...,...,...,...
896,Sci-Fi|Thriller,2688632,173961.0,30330,257.5,102043037879
897,Sci-Fi|Thriller|IMAX,715760,16088.0,3011,28.0,11080726300
898,Thriller,6309611,1420261.0,243568,2488.0,754255553721
899,War,41761,25535.0,5215,50.5,13176298237


In [29]:
genrecat = genres[genres['rating']>1000]

In [30]:
genrecat

,genres,movieId,year,userId,rating,timestamp
2,Action|Adventure,2889493,1073155.0,182730,2046.0,620972100175
80,Action|Adventure|Drama,5275641,934275.0,159508,1738.0,541773828187
98,Action|Adventure|Fantasy,5323481,981931.0,162942,1587.5,554137146344
117,Action|Adventure|Sci-Fi,25263186,4270982.0,706858,7689.0,2446931166312
118,Action|Adventure|Sci-Fi|IMAX,31168987,1030311.0,171801,1769.0,690238877719
...,...,...,...,...,...,...
870,Horror|Sci-Fi|Thriller,3261894,629377.0,109021,1003.5,357014834973
871,Horror|Thriller,8533079,1163059.0,192438,1687.5,659575749918
885,Mystery|Sci-Fi|Thriller,3913003,537111.0,92781,1047.0,296889224612
887,Mystery|Thriller,3244804,1127924.0,190879,2256.5,635930993955


In [31]:
genrecat['average'] = genrecat['rating'].mean()

In [32]:
genrecat

,genres,movieId,year,userId,rating,timestamp,average
2,Action|Adventure,2889493,1073155.0,182730,2046.0,620972100175,3498.592857
80,Action|Adventure|Drama,5275641,934275.0,159508,1738.0,541773828187,3498.592857
98,Action|Adventure|Fantasy,5323481,981931.0,162942,1587.5,554137146344,3498.592857
117,Action|Adventure|Sci-Fi,25263186,4270982.0,706858,7689.0,2446931166312,3498.592857
118,Action|Adventure|Sci-Fi|IMAX,31168987,1030311.0,171801,1769.0,690238877719,3498.592857
...,...,...,...,...,...,...,...
870,Horror|Sci-Fi|Thriller,3261894,629377.0,109021,1003.5,357014834973,3498.592857
871,Horror|Thriller,8533079,1163059.0,192438,1687.5,659575749918,3498.592857
885,Mystery|Sci-Fi|Thriller,3913003,537111.0,92781,1047.0,296889224612,3498.592857
887,Mystery|Thriller,3244804,1127924.0,190879,2256.5,635930993955,3498.592857


In [33]:
genrecat['sd'] = genrecat['rating'].std()

In [34]:
genrecat

,genres,movieId,year,userId,rating,timestamp,average,sd
2,Action|Adventure,2889493,1073155.0,182730,2046.0,620972100175,3498.592857,4767.749766
80,Action|Adventure|Drama,5275641,934275.0,159508,1738.0,541773828187,3498.592857,4767.749766
98,Action|Adventure|Fantasy,5323481,981931.0,162942,1587.5,554137146344,3498.592857,4767.749766
117,Action|Adventure|Sci-Fi,25263186,4270982.0,706858,7689.0,2446931166312,3498.592857,4767.749766
118,Action|Adventure|Sci-Fi|IMAX,31168987,1030311.0,171801,1769.0,690238877719,3498.592857,4767.749766
...,...,...,...,...,...,...,...,...
870,Horror|Sci-Fi|Thriller,3261894,629377.0,109021,1003.5,357014834973,3498.592857,4767.749766
871,Horror|Thriller,8533079,1163059.0,192438,1687.5,659575749918,3498.592857,4767.749766
885,Mystery|Sci-Fi|Thriller,3913003,537111.0,92781,1047.0,296889224612,3498.592857,4767.749766
887,Mystery|Thriller,3244804,1127924.0,190879,2256.5,635930993955,3498.592857,4767.749766
